<a href="https://colab.research.google.com/github/Rishabh5752/textattack/blob/main/TextAttack_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 117.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97

In [2]:
# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.


In [3]:
from textattack.transformations import WordSwap

class BananaWordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        return ['banana']

In [4]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack

# We're going to use our Banana word swap class as the attack transformation.
transformation = BananaWordSwap()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

In [5]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [6]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=10)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

#The following legacy tutorial code shows how the Attack API works in detail.

#logger = CSVLogger(color_method='html')

#num_successes = 0
#i = 0
#while num_successes < 10:
    #result = next(results_iterable)
#    example, ground_truth_output = dataset[i]
#    i += 1
#    result = attack.attack(example, ground_truth_output)
#    if isinstance(result, SuccessfulAttackResult):
#        logger.log_attack_result(result)
#        num_successes += 1
#       print(f'{num_successes} of 10 successes complete.')

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



 10%|█         | 1/10 [00:06<00:57,  6.41s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:07<01:03,  7.04s/it]

[[Business (100%)]] --> [[World (89%)]]

Fears for T N [[pension]] after [[talks]] [[Unions]] representing [[workers]] at Turner   Newall say they are '[[disappointed']] after talks with stricken parent firm Federal [[Mogul]].

Fears for T N [[banana]] after [[banana]] [[banana]] representing [[banana]] at Turner   Newall say they are '[[banana]] after talks with stricken parent firm Federal [[banana]].




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

FOAFKey: FOAF, PGP, Key Distribution, and Bloom Filters \\FOAF/LOAF  and bloom filters have a lot of interesting properties for social\network and whitelist distribution.\\I think we can go one level higher though and include GPG/OpenPGP key\fingerpring distribution in the FOAF file for simple web-of-trust based key\distribution.\\What if we used FOAF and included the PGP key fingerprint(s) for identities?\This could mean a lot.  You include the PGP key fingerprints within the FOAF\file of your direct friends and then include a bloom filter of the PGP key\fingerprints of your entire whitelist (the source FOAF file would of course need\to be encrypted ).\\Your whitelist would be populated from the social network as your client\discovered new identit ...\\




[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [05:30<00:36, 36.69s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (100%)]]

[[E-mail]] scam targets police chief Wiltshire Police warns about "[[phishing]]" after its fraud squad chief was targeted.

[[banana]] scam targets police chief Wiltshire Police warns about "[[banana]]" after its fraud squad chief was targeted.




[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [05:31<00:00, 33.12s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (77%)]]

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[card]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[banana]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 18.71% |
| Average num. words per input: | 63.0   |
| Avg num queries:     

In [9]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.dataframe[['original_text', 'perturbed_text']].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 10>:10                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'CSVLogger' object has no attribute 'dataframe'

In [10]:
# For AG News, labels are 0: World, 1: Sports, 2: Business, 3: Sci/Tech

custom_dataset = [
    ('Malaria deaths in Africa fall by 5% from last year', 0),
    ('Washington Nationals defeat the Houston Astros to win the World Series', 1),
    ('Exxon Mobil hires a new CEO', 2),
    ('Microsoft invests $1 billion in OpenAI', 3),
]

attack_args = AttackArgs(num_examples=4)

dataset = Dataset(custom_dataset)

attacker = Attacker(attack, dataset, attack_args)

results_iterable = attacker.attack_dataset()

logger = CSVLogger(color_method='html')

for result in results_iterable:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML

display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  25%|██▌       | 1/4 [00:00<00:01,  2.73it/s]textattack: CSVLogger exiting without calling flush().


--------------------------------------------- Result 1 ---------------------------------------------
[[0 (96%)]] --> [[3 (80%)]]

Malaria [[deaths]] in Africa fall by 5% from last year

Malaria [[banana]] in Africa fall by 5% from last year




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  50%|█████     | 2/4 [00:01<00:01,  1.75it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (98%)]] --> [[3 (87%)]]

[[Washington]] [[Nationals]] [[defeat]] the Houston Astros to win the World Series

[[banana]] [[banana]] [[banana]] the Houston Astros to win the World Series




...

--------------------------------------------- Result 3 ---------------------------------------------
[[2 (99%)]] --> [[3 (94%)]]

[[Exxon]] [[Mobil]] [[hires]] a new [[CEO]]

[[banana]] [[banana]] [[banana]] a new [[banana]]




[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4: 100%|██████████| 4/4 [00:01<00:00,  2.04it/s]

--------------------------------------------- Result 4 ---------------------------------------------
[[3 (93%)]] --> [[2 (100%)]]

[[Microsoft]] invests $1 billion in OpenAI

[[banana]] invests $1 billion in OpenAI



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 30.15% |
| Average num. words per input: | 8.25   |
| Avg num queries:              | 12.75  |
+-------------------------------+--------+


textattack: Logging to CSV at path results.csv


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 25>:25                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'CSVLogger' object has no attribute 'df'